In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip install mne

     |████████████████████████████████| 7.4 MB 4.1 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
# localized fetch_data function by using mne library
# https://github.com/mne-tools/mne-python/blob/maint/0.20/mne/datasets/sleep_physionet/age.py#L18-L111
data_path = _data_path
BASE_URL = 'https://physionet.org/physiobank/database/sleep-edfx/sleep-cassette/'
def fetch_data(subjects, recording=[1, 2], path=None, force_update=False,
               update_path=None, base_url=BASE_URL,
               verbose=None):  # noqa: D301
    records = np.loadtxt(AGE_SLEEP_RECORDS,
                         skiprows=1,
                         delimiter=',',
                         usecols=(0, 1, 2, 6, 7),
                         dtype={'names': ('subject', 'record', 'type', 'sha',
                                          'fname'),
                                'formats': ('<i2', 'i1', '<S9', 'S40', '<S22')}
                         )
    psg_records = records[np.where(records['type'] == b'PSG')]
    hyp_records = records[np.where(records['type'] == b'Hypnogram')]

    path = data_path(path=path, update_path=update_path)
    params = [path, force_update, base_url]
    fnames = []
    for subject in subjects:
        for idx in np.where(psg_records['subject'] == subject)[0]:
            if psg_records['record'][idx] in recording:
                psg_fname = _fetch_one(psg_records['fname'][idx].decode(),
                                       psg_records['sha'][idx].decode(),
                                       *params)
                hyp_fname = _fetch_one(hyp_records['fname'][idx].decode(),
                                       hyp_records['sha'][idx].decode(),
                                       *params)
                fnames.append([psg_fname, hyp_fname])

    return fnames

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
npz_files = sorted(glob.glob("/gdrive/My Drive/NPZ_files/***.npz"))
X = np.zeros((0, 3000, 1))
y = []
for fn in tqdm(npz_files[:number_of_files]):
    samples = np.load(fn)
    X_data = samples['x']
    X = np.concatenate((X, X_data), axis=0)
    y.extend(samples['y'])
y = np.array(y)

#sorted(glob.glob("/gdrive/My Drive/NPZ_files/***.npz"))


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
npz_files

['/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4092E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4101E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4102E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4

In [ ]:
len(npz_files[:number_of_files])

100

In [ ]:
npz_files[:number_of_files]

['/gdrive/My Drive/NPZ_files/SC4001E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4002E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4011E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4012E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4021E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4022E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4031E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4032E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4041E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4042E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4051E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4052E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4061E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4062E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4071E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4072E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4081E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4082E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4091E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4092E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4101E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4102E0.npz',
 '/gdrive/My Drive/NPZ_files/SC4

In [ ]:
npz_files[148:150]

['/gdrive/My Drive/NPZ_files/SC4821G0.npz',
 '/gdrive/My Drive/NPZ_files/SC4822G0.npz']

In [ ]:
X

array([[[ 8.11135578e+00],
        [ 1.74886456e+01],
        [ 2.12395611e+01],
        ...,
        [-1.03619051e+01],
        [-1.11120882e+01],
        [-2.10989022e+00]],

       [[-1.07369967e+01],
        [-1.13934069e+01],
        [-4.45421267e+00],
        ...,
        [ 5.88424911e+01],
        [ 4.83399277e+01],
        [ 5.36849823e+01]],

       [[ 6.13743591e+01],
        [ 3.86813202e+01],
        [ 4.95589752e+01],
        ...,
        [ 3.36175842e+01],
        [ 3.43677673e+01],
        [ 3.18358974e+01]],

       ...,

       [[ 3.47106218e+00],
        [-5.03052510e-02],
        [ 2.56556773e+00],
        ...,
        [ 9.55799758e-01],
        [-1.86129427e+00],
        [ 3.37045169e+00]],

       [[ 2.26373625e+00],
        [ 3.52136761e-01],
        [ 2.86739922e+00],
        ...,
        [ 2.54041519e+01],
        [ 2.02730160e+01],
        [ 1.82608051e+01]],

       [[ 1.96693535e+01],
        [ 1.65504265e+01],
        [ 1.54437122e+01],
        ...,
        

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
from keras.utils import np_utils

In [ ]:
if VBS:
    print("Shape of the input data: {}".format(X.shape))
    print("Shape of the sleep stages: {}".format(y.shape))


Shape of the input data: (116718, 3000, 1)
Shape of the sleep stages: (116718,)


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
def butter_bandpass(lowcut, highpass, fs, order=4):
    nyq = 0.5 * fs
    #       low = lowcut / nyq
    high = highpass / nyq
    b, a = butter(order, high, btype='highpass')
    return b, a

In [ ]:
def butter_bandpass_filter(data, highpass, fs, order=4):
    b, a = butter_bandpass(0, highpass, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
#npz_files = sorted(glob.glob(os.path.join(output_path, "*.npz")))
X_sub_482 = np.zeros((0, 3000, 1))
y_sub_482 = []
for fn in tqdm(npz_files[148:150]): #extracting data from the npz files corresponding to both tge recording 1, 2 of the 50th subject (no 53rd subject)
    samples = np.load(fn)
    X_data = samples['x']
    X_sub_482 = np.concatenate((X_sub_482, X_data), axis=0)
    y_sub_482.extend(samples['y'])
y_sub_482 = np.array(y_sub_482)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
X_sub_482

array([[[-49.10647202],
        [-48.57606888],
        [-45.21685028],
        ...,
        [  1.81220996],
        [ -2.51941395],
        [  3.75702071]],

       [[ 17.63589668],
        [  4.37582397],
        [ 25.59194183],
        ...,
        [ 52.20048904],
        [ 55.55970764],
        [ 57.06251526]],

       [[ 50.25567627],
        [ 55.64810562],
        [ 50.16727829],
        ...,
        [-13.03907204],
        [-16.75189209],
        [-13.12747288]],

       ...,

       [[  4.04444456],
        [  2.26666665],
        [  6.44444466],
        ...,
        [ -6.17777777],
        [ 10.17777824],
        [-48.93333435]],

       [[-43.95555496],
        [-63.42222214],
        [-70.8888855 ],
        ...,
        [ 20.66666603],
        [ 14.88888931],
        [ 22.35555649]],

       [[ 21.73333359],
        [ 21.55555534],
        [ 21.91111183],
        ...,
        [  1.28888893],
        [  2.97777772],
        [  5.28888893]]])

In [ ]:
y_sub_482

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
if VBS:
    print("Shape of the subject(482) data: {}".format(X_sub_482.shape))
    print("Shape of the corresponding sleep stage: {}".format(y_sub_482.shape))

Shape of the subject(482) data: (3070, 3000, 1)
Shape of the corresponding sleep stage: (3070,)


In [ ]:
pp_x = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X])

if VBS:
    print(pp_x.shape)

(116718, 3000, 1)


In [ ]:
y.shape

(116718,)

In [ ]:
type(y)

numpy.ndarray

In [ ]:
pp_x

array([[[ 3.91319654e-02],
        [ 8.43712310e-02],
        [ 1.02466935e-01],
        ...,
        [-4.99893880e-02],
        [-5.36085289e-02],
        [-1.01788349e-02]],

       [[-5.17989585e-02],
        [-5.49657067e-02],
        [-2.14886513e-02],
        ...,
        [ 2.83876381e-01],
        [ 2.33208409e-01],
        [ 2.58994788e-01]],

       [[ 2.96090982e-01],
        [ 1.86611970e-01],
        [ 2.39089513e-01],
        ...,
        [ 1.62182769e-01],
        [ 1.65801910e-01],
        [ 1.53587300e-01]],

       ...,

       [[ 1.67455958e-02],
        [-2.42689804e-04],
        [ 1.23771796e-02],
        ...,
        [ 4.61110622e-03],
        [-8.97952265e-03],
        [ 1.62602162e-02]],

       [[ 1.09210409e-02],
        [ 1.69882864e-03],
        [ 1.38333184e-02],
        ...,
        [ 1.22558352e-01],
        [ 9.78039900e-02],
        [ 8.80963939e-02]],

       [[ 9.48917148e-02],
        [ 7.98449401e-02],
        [ 7.45057705e-02],
        ...,
        

In [ ]:
pp_x_rshp=pp_x.reshape(116718, 3000)

In [ ]:
pp_x_rshp_df=pd.DataFrame(pp_x_rshp)

In [ ]:
pp_x_rshp_df['sleep_stage']=y

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,0.014250,0.022393,0.032346,0.012893,-0.001131,0.002488,0.014250,0.016060,-0.007917,-0.029179,-0.005655,-0.004750,-0.031441,-0.062656,-0.039132,-0.036418,-0.068537,-0.028275,-0.024655,-0.060394,-0.079847,-0.056323,-0.058132,-0.052704,-0.078943,-0.064014,-0.026917,-0.046370,-0.021941,-0.024203,...,-0.060847,-0.072157,-0.041846,-0.006107,-0.024203,-0.041846,-0.041846,0.011536,-0.029179,-0.034608,-0.052251,-0.030989,-0.072609,-0.046370,-0.013346,-0.064918,-0.052251,-0.065371,-0.061299,-0.062204,-0.060394,-0.002941,-0.030537,-0.049085,-0.016965,-0.031441,-0.001583,-0.040037,-0.047275,-0.064014,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179,0
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,-0.050894,-0.051799,-0.083919,-0.059942,-0.064014,-0.054513,-0.058132,0.014250,-0.020131,0.000679,-0.024203,-0.057228,-0.007012,-0.074419,-0.002036,-0.078490,-0.034156,-0.035965,0.007464,-0.007464,-0.024655,0.013798,-0.007012,-0.028275,-0.007012,-0.030989,-0.007464,-0.026465,0.003845,0.046823,...,0.233208,0.188874,0.187969,0.217827,0.143635,0.224613,0.137301,0.191588,0.216922,0.191136,0.163088,0.206517,0.210136,0.128253,0.163540,0.126444,0.154944,0.154944,0.197922,0.206970,0.216922,0.194303,0.187517,0.183898,0.220994,0.242256,0.201993,0.196112,0.280257,0.260352,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995,0
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,0.222803,0.214208,0.140920,0.119658,0.040037,0.084824,0.027822,0.106086,0.065823,0.029632,0.003845,-0.008369,-0.042299,-0.088895,-0.108800,-0.125087,-0.071252,-0.050442,-0.051799,-0.044108,-0.002941,-0.065823,0.011084,-0.144087,-0.082109,-0.157659,-0.102467,-0.075776,-0.040489,-0.049989,...,-0.292472,-0.290210,-0.287496,-0.278900,-0.269852,-0.251757,-0.232304,-0.209232,-0.159468,-0.101562,-0.076681,-0.028727,-0.022393,0.025108,0.066728,0.099753,0.081204,0.130063,0.168516,0.180731,0.187064,0.198827,0.177564,0.185255,0.197017,0.231851,0.224161,0.241804,0.227327,0.187517,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587,0
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,-0.006107,-0.003393,-0.010631,-0.007464,-0.050894,-0.029632,0.006107,-0.018774,0.021941,0.006107,0.016060,0.014250,-0.020131,0.014250,-0.001583,-0.029179,-0.039132,0.041846,-0.112420,-0.025108,-0.010179,0.013346,0.031894,0.007012,-0.031441,-0.010631,-0.026465,-0.015155,-0.005655,-0.018322,...,-0.189779,-0.235470,-0.254923,-0.281162,-0.270757,-0.300163,-0.300163,-0.291115,-0.282067,-0.280257,-0.270757,-0.259900,-0.251304,-0.238637,-0.238637,-0.234113,-0.224613,-0.208779,-0.199731,-0.202446,-0.195660,-0.191588,-0.177564,-0.140468,-0.157659,-0.152683,-0.155849,-0.144087,-0.124182,-0.100657,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829,0
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,0.351283,0.387022,0.396522,0.361688,0.324592,0.347664,0.241804,0.263519,0.259447,0.206065,0.225970,0.163088,0.125539,0.238637,0.009726,0.205612,0.039132,-0.016060,0.003393,-0.053609,-0.000226,0.359426,0.334997,0.049537,-0.149516,0.057228,0.016060,-0.051347,-0.035965,0.048632,...,-0.060847,-0.064014,-0.085276,-0.155397,-0.178017,-0.152230,-0.112872,-0.124634,-0.081204,-0.028275,-0.050442,-0.062204,-0.092967,-0.075323,-0.036870,-0.109705,-0.088443,-0.143182,-0.242709,-0.293377,-0.351735,-0.342687,-0.359426,-0.343592,-0.339521,-0.

In [ ]:
dt_shuf3=pp_x_rshp_df.sample(frac=1,replace=True, random_state=789)

In [ ]:
dt_shuf3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
4659,0.012246,0.018036,0.009129,-0.004231,-0.020262,-0.020708,-0.027833,-0.039411,-0.059451,-0.067467,-0.071029,-0.086170,-0.100421,-0.109327,-0.118234,-0.123132,-0.136046,-0.146734,-0.145844,-0.148070,-0.143172,-0.146289,-0.148516,-0.134711,-0.126695,-0.118234,-0.102647,-0.090623,-0.066131,-0.059896,-0.063013,-0.076373,-0.070139,-0.068357,-0.075482,-0.072365,-0.063459,-0.050990,-0.050544,-0.049208,...,-0.046091,-0.023825,-0.003340,0.009129,0.008238,-0.009574,-0.026051,-0.043864,-0.032286,-0.009574,-0.002004,-0.007348,-0.004231,-0.006903,-0.018926,-0.020262,-0.005567,0.012246,0.023380,0.022934,0.030505,0.019817,0.015809,0.024270,0.029169,0.032731,0.034513,0.012246,0.010910,0.011801,0.023380,0.029169,0.036294,0.031841,0.037185,0.032286,0.036739,0.044755,0.066131,2
83598,-0.011252,0.008653,0.001603,-0.019546,-0.037378,0.000774,-0.009593,0.019850,0.000359,-0.030743,-0.030743,-0.020790,0.007409,-0.085482,-0.029499,-0.030328,-0.040281,-0.039866,-0.040695,-0.042769,-0.026181,-0.064333,-0.059357,-0.043183,-0.017887,-0.000470,-0.028255,-0.040281,-0.015399,-0.039451,-0.014570,-0.022449,-0.019131,-0.018717,-0.038622,-0.029499,0.019021,-0.044428,-0.034890,0.012385,...,-0.009593,-0.045257,-0.027010,-0.013326,-0.056039,-0.028669,-0.012081,-0.045257,-0.056868,-0.043183,-0.031572,-0.008764,-0.003373,-0.033646,0.003262,0.031461,0.036852,0.052611,0.012385,0.034364,0.048049,0.047634,0.028973,-0.015399,0.001603,0.032705,0.029803,-0.003788,0.067540,0.081225,-0.010837,0.036023,-0.021619,0.058002,0.036438,0.050123,0.021094,-0.000470,0.008238,0
96670,0.083314,0.118263,0.099761,0.100994,0.088660,0.120730,0.105928,0.089071,0.096060,0.095649,0.105106,0.060290,0.086193,0.029453,0.034798,-0.001795,-0.027698,-0.038389,-0.074982,-0.113220,-0.132133,-0.120209,-0.118976,-0.114864,-0.114042,-0.113631,-0.103352,-0.095540,-0.086494,-0.069225,-0.086494,-0.062647,-0.026465,-0.003029,0.016296,0.003961,0.013006,0.050833,0.032331,0.005605,...,0.041376,0.031097,-0.000562,0.021229,-0.006729,0.006839,0.002316,0.000260,0.004783,0.009717,0.035620,0.058234,0.049599,0.078792,0.064401,0.056589,0.069335,0.076736,0.031097,0.038909,-0.016597,-0.017831,-0.076215,-0.101296,-0.138711,-0.158858,-0.215187,-0.232867,-0.264526,-0.312632,-0.308520,-0.328256,-0.344291,-0.330311,-0.347169,-0.298241,-0.327844,-0.303997,-0.296596,3
45266,-0.006153,0.002181,0.012830,0.021626,0.041072,0.053573,0.051721,0.053573,0.053110,0.054962,0.043387,0.042461,0.030423,0.024867,0.024404,0.019312,0.022552,0.022552,0.026719,0.015145,-0.002912,-0.004764,-0.008005,-0.017728,-0.024673,-0.022821,-0.032081,-0.029303,-0.036248,-0.042267,-0.047360,-0.058472,-0.056620,-0.053379,-0.043656,-0.042730,-0.031155,-0.022358,-0.000134,0.009126,...,-0.013561,-0.018191,-0.009857,-0.020506,-0.021432,-0.012172,-0.014487,-0.017265,-0.025599,-0.025136,-0.016339,-0.008005,-0.009394,-0.014487,-0.007079,-0.003838,0.004959,0.005422,-0.000134,0.002644,-0.000134,0.010515,0.003570,0.007737,0.016534,0.013293,0.012367,0.017460,0.021626,0.030886,0.033201,0.022552,0.018386,0.024404,0.019775,0.009589,-0.000134,0.009126,0.016071,4
71553,-0.052897,-0.055795,-0.033093,-0.021500,0.002168,0.016658,-0.023916,-0.032610,-0.047101,-0.033093,-0.032610,-0.076565,-0.112309,-0.112309,-0.097335,-0.066905,-0.076082,-0.084776,-0.063040,-0.052897,-0.072701,-0.072218,-0.075116,-0.077048,-0.071252,-0.051931,-0.077048,-0.107479,-0.104580,-0.097335,-0.103614,-0.103614,-0.103131,-0.078980,-0.064006,-0.068354,-0.062074,-0.049033,-0.000730,0.031149,...,-0.007493,-0.022466,-0.037923,-0.027780,-0.063523,-0.014738,-0.012323,-0.008459,-0.013772,-0.035025,-0.027780,0.016175,0.028251,0.043225,0.066893,0.081384,0.095874,0.084282,0.096357,0.081867,0.08911

In [ ]:
dt_shuf3['sleep_stage']

4659     2
83598    0
96670    3
45266    4
71553    2
        ..
18162    4
55862    4
75072    2
18581    2
45137    2
Name: sleep_stage, Length: 116718, dtype: int32

In [ ]:
x_shuf3=dt_shuf3.drop('sleep_stage',axis=1)
y_shuf3=dt_shuf3['sleep_stage']

In [ ]:
x_shuf3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
4659,0.012246,0.018036,0.009129,-0.004231,-0.020262,-0.020708,-0.027833,-0.039411,-0.059451,-0.067467,-0.071029,-0.086170,-0.100421,-0.109327,-0.118234,-0.123132,-0.136046,-0.146734,-0.145844,-0.148070,-0.143172,-0.146289,-0.148516,-0.134711,-0.126695,-0.118234,-0.102647,-0.090623,-0.066131,-0.059896,-0.063013,-0.076373,-0.070139,-0.068357,-0.075482,-0.072365,-0.063459,-0.050990,-0.050544,-0.049208,...,-0.023825,-0.046091,-0.023825,-0.003340,0.009129,0.008238,-0.009574,-0.026051,-0.043864,-0.032286,-0.009574,-0.002004,-0.007348,-0.004231,-0.006903,-0.018926,-0.020262,-0.005567,0.012246,0.023380,0.022934,0.030505,0.019817,0.015809,0.024270,0.029169,0.032731,0.034513,0.012246,0.010910,0.011801,0.023380,0.029169,0.036294,0.031841,0.037185,0.032286,0.036739,0.044755,0.066131
83598,-0.011252,0.008653,0.001603,-0.019546,-0.037378,0.000774,-0.009593,0.019850,0.000359,-0.030743,-0.030743,-0.020790,0.007409,-0.085482,-0.029499,-0.030328,-0.040281,-0.039866,-0.040695,-0.042769,-0.026181,-0.064333,-0.059357,-0.043183,-0.017887,-0.000470,-0.028255,-0.040281,-0.015399,-0.039451,-0.014570,-0.022449,-0.019131,-0.018717,-0.038622,-0.029499,0.019021,-0.044428,-0.034890,0.012385,...,-0.057283,-0.009593,-0.045257,-0.027010,-0.013326,-0.056039,-0.028669,-0.012081,-0.045257,-0.056868,-0.043183,-0.031572,-0.008764,-0.003373,-0.033646,0.003262,0.031461,0.036852,0.052611,0.012385,0.034364,0.048049,0.047634,0.028973,-0.015399,0.001603,0.032705,0.029803,-0.003788,0.067540,0.081225,-0.010837,0.036023,-0.021619,0.058002,0.036438,0.050123,0.021094,-0.000470,0.008238
96670,0.083314,0.118263,0.099761,0.100994,0.088660,0.120730,0.105928,0.089071,0.096060,0.095649,0.105106,0.060290,0.086193,0.029453,0.034798,-0.001795,-0.027698,-0.038389,-0.074982,-0.113220,-0.132133,-0.120209,-0.118976,-0.114864,-0.114042,-0.113631,-0.103352,-0.095540,-0.086494,-0.069225,-0.086494,-0.062647,-0.026465,-0.003029,0.016296,0.003961,0.013006,0.050833,0.032331,0.005605,...,0.022463,0.041376,0.031097,-0.000562,0.021229,-0.006729,0.006839,0.002316,0.000260,0.004783,0.009717,0.035620,0.058234,0.049599,0.078792,0.064401,0.056589,0.069335,0.076736,0.031097,0.038909,-0.016597,-0.017831,-0.076215,-0.101296,-0.138711,-0.158858,-0.215187,-0.232867,-0.264526,-0.312632,-0.308520,-0.328256,-0.344291,-0.330311,-0.347169,-0.298241,-0.327844,-0.303997,-0.296596
45266,-0.006153,0.002181,0.012830,0.021626,0.041072,0.053573,0.051721,0.053573,0.053110,0.054962,0.043387,0.042461,0.030423,0.024867,0.024404,0.019312,0.022552,0.022552,0.026719,0.015145,-0.002912,-0.004764,-0.008005,-0.017728,-0.024673,-0.022821,-0.032081,-0.029303,-0.036248,-0.042267,-0.047360,-0.058472,-0.056620,-0.053379,-0.043656,-0.042730,-0.031155,-0.022358,-0.000134,0.009126,...,-0.014950,-0.013561,-0.018191,-0.009857,-0.020506,-0.021432,-0.012172,-0.014487,-0.017265,-0.025599,-0.025136,-0.016339,-0.008005,-0.009394,-0.014487,-0.007079,-0.003838,0.004959,0.005422,-0.000134,0.002644,-0.000134,0.010515,0.003570,0.007737,0.016534,0.013293,0.012367,0.017460,0.021626,0.030886,0.033201,0.022552,0.018386,0.024404,0.019775,0.009589,-0.000134,0.009126,0.016071
71553,-0.052897,-0.055795,-0.033093,-0.021500,0.002168,0.016658,-0.023916,-0.032610,-0.047101,-0.033093,-0.032610,-0.076565,-0.112309,-0.112309,-0.097335,-0.066905,-0.076082,-0.084776,-0.063040,-0.052897,-0.072701,-0.072218,-0.075116,-0.077048,-0.071252,-0.051931,-0.077048,-0.107479,-0.104580,-0.097335,-0.103614,-0.103614,-0.103131,-0.078980,-0.064006,-0.068354,-0.062074,-0.049033,-0.000730,0.031149,...,-0.000247,-0.007493,-0.022466,-0.037923,-0.027780,-0.063523,-0.014738,-0.012323,-0.008459,-0.013772,-0.035025,-0.027780,0.016175,0.028251,0.043225,0.066893,0.081384,0.095874,

In [ ]:
y_shuf3_=np_utils.to_categorical(y_shuf3)

In [ ]:
x_shuf3_rshp=np.array(x_shuf3).reshape(116718,3000,1)

In [ ]:
pp_X_sub_482 = np.array([butter_bandpass_filter(sample, highpass=40.0, fs=100, order=4) for sample in X_sub_482])

if VBS:
    print(pp_X_sub_482.shape)

(3070, 3000, 1)


In [ ]:
pp_X_sub_482

array([[[-0.23690648],
        [-0.23434764],
        [-0.21814161],
        ...,
        [ 0.00874272],
        [-0.01215452],
        [ 0.01812516]],

       [[ 0.08508162],
        [ 0.02111048],
        [ 0.12346431],
        ...,
        [ 0.25183308],
        [ 0.26803911],
        [ 0.27528917]],

       [[ 0.24245064],
        [ 0.26846557],
        [ 0.24202418],
        ...,
        [-0.06290496],
        [-0.08081688],
        [-0.06333144]],

       ...,

       [[ 0.01951179],
        [ 0.01093518],
        [ 0.03109021],
        ...,
        [-0.02980372],
        [ 0.0491011 ],
        [-0.23607121]],

       [[-0.21205669],
        [-0.30597058],
        [-0.34199232],
        ...,
        [ 0.09970309],
        [ 0.07182911],
        [ 0.10785088]],

       [[ 0.10484906],
        [ 0.1039914 ],
        [ 0.10570673],
        ...,
        [ 0.00621804],
        [ 0.01436582],
        [ 0.02551542]]])

The following corresponds to reshaping the existing 3d data to 2d and converting it to .csv. The csv file was created on google drive and later downloaded to the local pc

In [ ]:
pp_x_rshp=pp_x.reshape(189560,3000)

In [ ]:
pp_x_rshp

array([[ 0.03913197,  0.08437123,  0.10246694, ..., -0.04998939,
        -0.05360853, -0.01017883],
       [-0.05179896, -0.05496571, -0.02148865, ...,  0.28387638,
         0.23320841,  0.25899479],
       [ 0.29609098,  0.18661197,  0.23908951, ...,  0.16218277,
         0.16580191,  0.1535873 ],
       ...,
       [-0.03455384,  0.00189911,  0.05185315, ..., -0.0525553 ,
         0.025301  ,  0.01360005],
       [-0.00620155,  0.02170071, -0.06515632, ...,  0.0352018 ,
         0.0203506 ,  0.0437525 ],
       [ 0.12205883, -0.08855822, -0.07505712, ..., -0.06470629,
        -0.08810818, -0.10835982]])

In [ ]:
pp_X_sub_482_rshp=pp_X_sub_482.reshape(3070,3000)

In [ ]:
pp_x_rshp_df=pd.DataFrame(pp_x_rshp)

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,0.014250,0.022393,0.032346,0.012893,-0.001131,0.002488,0.014250,0.016060,-0.007917,-0.029179,-0.005655,-0.004750,-0.031441,-0.062656,-0.039132,-0.036418,-0.068537,-0.028275,-0.024655,-0.060394,-0.079847,-0.056323,-0.058132,-0.052704,-0.078943,-0.064014,-0.026917,-0.046370,-0.021941,-0.024203,...,-0.061752,-0.060847,-0.072157,-0.041846,-0.006107,-0.024203,-0.041846,-0.041846,0.011536,-0.029179,-0.034608,-0.052251,-0.030989,-0.072609,-0.046370,-0.013346,-0.064918,-0.052251,-0.065371,-0.061299,-0.062204,-0.060394,-0.002941,-0.030537,-0.049085,-0.016965,-0.031441,-0.001583,-0.040037,-0.047275,-0.064014,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,-0.050894,-0.051799,-0.083919,-0.059942,-0.064014,-0.054513,-0.058132,0.014250,-0.020131,0.000679,-0.024203,-0.057228,-0.007012,-0.074419,-0.002036,-0.078490,-0.034156,-0.035965,0.007464,-0.007464,-0.024655,0.013798,-0.007012,-0.028275,-0.007012,-0.030989,-0.007464,-0.026465,0.003845,0.046823,...,0.248137,0.233208,0.188874,0.187969,0.217827,0.143635,0.224613,0.137301,0.191588,0.216922,0.191136,0.163088,0.206517,0.210136,0.128253,0.163540,0.126444,0.154944,0.154944,0.197922,0.206970,0.216922,0.194303,0.187517,0.183898,0.220994,0.242256,0.201993,0.196112,0.280257,0.260352,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,0.222803,0.214208,0.140920,0.119658,0.040037,0.084824,0.027822,0.106086,0.065823,0.029632,0.003845,-0.008369,-0.042299,-0.088895,-0.108800,-0.125087,-0.071252,-0.050442,-0.051799,-0.044108,-0.002941,-0.065823,0.011084,-0.144087,-0.082109,-0.157659,-0.102467,-0.075776,-0.040489,-0.049989,...,-0.284329,-0.292472,-0.290210,-0.287496,-0.278900,-0.269852,-0.251757,-0.232304,-0.209232,-0.159468,-0.101562,-0.076681,-0.028727,-0.022393,0.025108,0.066728,0.099753,0.081204,0.130063,0.168516,0.180731,0.187064,0.198827,0.177564,0.185255,0.197017,0.231851,0.224161,0.241804,0.227327,0.187517,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,-0.006107,-0.003393,-0.010631,-0.007464,-0.050894,-0.029632,0.006107,-0.018774,0.021941,0.006107,0.016060,0.014250,-0.020131,0.014250,-0.001583,-0.029179,-0.039132,0.041846,-0.112420,-0.025108,-0.010179,0.013346,0.031894,0.007012,-0.031441,-0.010631,-0.026465,-0.015155,-0.005655,-0.018322,...,-0.166254,-0.189779,-0.235470,-0.254923,-0.281162,-0.270757,-0.300163,-0.300163,-0.291115,-0.282067,-0.280257,-0.270757,-0.259900,-0.251304,-0.238637,-0.238637,-0.234113,-0.224613,-0.208779,-0.199731,-0.202446,-0.195660,-0.191588,-0.177564,-0.140468,-0.157659,-0.152683,-0.155849,-0.144087,-0.124182,-0.100657,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,0.351283,0.387022,0.396522,0.361688,0.324592,0.347664,0.241804,0.263519,0.259447,0.206065,0.225970,0.163088,0.125539,0.238637,0.009726,0.205612,0.039132,-0.016060,0.003393,-0.053609,-0.000226,0.359426,0.334997,0.049537,-0.149516,0.057228,0.016060,-0.051347,-0.035965,0.048632,...,-0.050894,-0.060847,-0.064014,-0.085276,-0.155397,-0.178017,-0.152230,-0.112872,-0.124634,-0.081204,-0.028275,-0.050442,-0.062204,-0.092967,-0.075323,-0.036870,-0.109705,-0.088443,-0.143182,-0.242709,-0.293377,-0.351735,-0.342687

In [ ]:
pp_x_rshp_df['sleep_stage']=y

In [ ]:
pp_x_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,0.039132,0.084371,0.102467,0.033251,0.045465,0.026465,0.008369,0.001131,-0.006560,0.009274,0.014250,0.022393,0.032346,0.012893,-0.001131,0.002488,0.014250,0.016060,-0.007917,-0.029179,-0.005655,-0.004750,-0.031441,-0.062656,-0.039132,-0.036418,-0.068537,-0.028275,-0.024655,-0.060394,-0.079847,-0.056323,-0.058132,-0.052704,-0.078943,-0.064014,-0.026917,-0.046370,-0.021941,-0.024203,...,-0.060847,-0.072157,-0.041846,-0.006107,-0.024203,-0.041846,-0.041846,0.011536,-0.029179,-0.034608,-0.052251,-0.030989,-0.072609,-0.046370,-0.013346,-0.064918,-0.052251,-0.065371,-0.061299,-0.062204,-0.060394,-0.002941,-0.030537,-0.049085,-0.016965,-0.031441,-0.001583,-0.040037,-0.047275,-0.064014,-0.034156,0.001583,-0.057228,-0.041394,-0.023751,0.002488,-0.049989,-0.053609,-0.010179,0
1,-0.051799,-0.054966,-0.021489,-0.001131,-0.016512,-0.114229,-0.056775,-0.085728,-0.113324,-0.078490,-0.050894,-0.051799,-0.083919,-0.059942,-0.064014,-0.054513,-0.058132,0.014250,-0.020131,0.000679,-0.024203,-0.057228,-0.007012,-0.074419,-0.002036,-0.078490,-0.034156,-0.035965,0.007464,-0.007464,-0.024655,0.013798,-0.007012,-0.028275,-0.007012,-0.030989,-0.007464,-0.026465,0.003845,0.046823,...,0.233208,0.188874,0.187969,0.217827,0.143635,0.224613,0.137301,0.191588,0.216922,0.191136,0.163088,0.206517,0.210136,0.128253,0.163540,0.126444,0.154944,0.154944,0.197922,0.206970,0.216922,0.194303,0.187517,0.183898,0.220994,0.242256,0.201993,0.196112,0.280257,0.260352,0.264876,0.197922,0.267138,0.264876,0.312377,0.251304,0.283876,0.233208,0.258995,0
2,0.296091,0.186612,0.239090,0.259900,0.302877,0.316901,0.272567,0.281614,0.220541,0.265328,0.222803,0.214208,0.140920,0.119658,0.040037,0.084824,0.027822,0.106086,0.065823,0.029632,0.003845,-0.008369,-0.042299,-0.088895,-0.108800,-0.125087,-0.071252,-0.050442,-0.051799,-0.044108,-0.002941,-0.065823,0.011084,-0.144087,-0.082109,-0.157659,-0.102467,-0.075776,-0.040489,-0.049989,...,-0.292472,-0.290210,-0.287496,-0.278900,-0.269852,-0.251757,-0.232304,-0.209232,-0.159468,-0.101562,-0.076681,-0.028727,-0.022393,0.025108,0.066728,0.099753,0.081204,0.130063,0.168516,0.180731,0.187064,0.198827,0.177564,0.185255,0.197017,0.231851,0.224161,0.241804,0.227327,0.187517,0.164445,0.146349,0.176207,0.176659,0.149516,0.163540,0.162183,0.165802,0.153587,0
3,0.134134,0.116491,0.116491,0.070347,0.066728,0.037322,0.030989,0.039584,0.023751,0.020131,-0.006107,-0.003393,-0.010631,-0.007464,-0.050894,-0.029632,0.006107,-0.018774,0.021941,0.006107,0.016060,0.014250,-0.020131,0.014250,-0.001583,-0.029179,-0.039132,0.041846,-0.112420,-0.025108,-0.010179,0.013346,0.031894,0.007012,-0.031441,-0.010631,-0.026465,-0.015155,-0.005655,-0.018322,...,-0.189779,-0.235470,-0.254923,-0.281162,-0.270757,-0.300163,-0.300163,-0.291115,-0.282067,-0.280257,-0.270757,-0.259900,-0.251304,-0.238637,-0.238637,-0.234113,-0.224613,-0.208779,-0.199731,-0.202446,-0.195660,-0.191588,-0.177564,-0.140468,-0.157659,-0.152683,-0.155849,-0.144087,-0.124182,-0.100657,-0.087086,-0.040942,0.009726,0.035060,0.129610,0.045465,0.126896,0.205612,0.274829,0
4,0.333640,0.359878,0.381141,0.386117,0.373450,0.372998,0.360783,0.367117,0.359878,0.382046,0.351283,0.387022,0.396522,0.361688,0.324592,0.347664,0.241804,0.263519,0.259447,0.206065,0.225970,0.163088,0.125539,0.238637,0.009726,0.205612,0.039132,-0.016060,0.003393,-0.053609,-0.000226,0.359426,0.334997,0.049537,-0.149516,0.057228,0.016060,-0.051347,-0.035965,0.048632,...,-0.060847,-0.064014,-0.085276,-0.155397,-0.178017,-0.152230,-0.112872,-0.124634,-0.081204,-0.028275,-0.050442,-0.062204,-0.092967,-0.075323,-0.036870,-0.109705,-0.088443,-0.143182,-0.242709,-0.293377,-0.351735,-0.342687,-0.359426,-0.343592,-0.339521,-0.

In [ ]:
pp_X_sub_482_rshp_df=pd.DataFrame(pp_X_sub_482_rshp)

In [ ]:
pp_X_sub_482_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.236906,-0.234348,-0.218142,-0.187435,-0.143082,-0.184024,-0.207053,-0.213024,-0.187009,-0.258230,-0.239465,-0.255245,-0.197244,-0.247568,-0.243304,-0.288936,-0.264627,-0.252686,-0.224965,-0.231362,-0.326466,-0.266760,-0.224965,-0.231789,-0.321348,-0.260363,-0.269745,-0.236054,-0.228803,-0.259510,-0.398967,-0.433085,-0.476159,-0.439482,-0.430526,-0.419437,-0.386172,-0.360584,-0.346937,-0.335849,...,-0.112376,-0.044140,-0.023243,-0.036037,-0.033478,0.050537,0.065037,0.020684,0.055655,0.043714,0.129861,0.042008,0.088067,0.100861,0.161847,0.123891,0.052670,0.092332,0.002346,0.081670,0.092758,0.110244,0.033478,0.045420,0.120905,-0.000213,0.012581,0.042008,-0.021537,-0.005331,0.036464,0.134979,0.138817,0.049258,0.100861,0.046272,0.065037,0.008743,-0.012155,0.018125
1,0.085082,0.021110,0.123464,0.145215,0.168244,0.133273,0.203641,0.077832,0.107258,0.140097,0.189568,0.091905,0.181891,0.143082,0.151185,0.194686,0.251407,-0.022816,0.083376,0.144362,-0.019405,-0.000213,0.049258,0.088067,0.101714,-0.020684,-0.009169,0.031772,-0.000213,-0.021963,-0.059920,-0.009596,-0.009169,-0.003199,0.030066,0.053096,0.060773,0.017699,0.070155,0.114935,...,0.216862,0.193406,0.164406,0.147347,0.240745,0.167391,0.169097,0.219421,0.253539,0.282966,0.194259,0.225392,0.224112,0.277422,0.278274,0.297892,0.256951,0.268892,0.321348,0.268039,0.268039,0.404937,0.175494,0.184877,0.080817,-0.077405,-0.132420,-0.193833,-0.084655,-0.110670,-0.075699,0.056934,0.222833,0.234348,0.151185,0.167818,0.214303,0.251833,0.268039,0.275289
2,0.242451,0.268466,0.242024,0.272730,0.204494,0.171656,0.211318,0.180185,0.187009,0.182318,0.187862,0.150332,0.161421,0.206200,0.166965,0.145215,0.123891,0.088493,0.026228,0.054375,-0.031346,-0.094038,-0.346084,-0.320922,-0.319216,-0.333716,-0.317084,-0.302584,-0.288936,-0.277422,-0.275289,-0.259083,-0.239039,-0.212171,-0.205347,-0.173362,-0.151185,-0.117920,-0.089773,-0.041581,...,0.031346,0.037317,0.060773,0.040302,0.039875,0.056508,0.045420,0.048831,0.091905,0.109391,0.122185,0.059067,0.149479,0.115361,0.118347,0.133273,0.118347,0.085082,0.067170,0.071861,0.059067,0.063758,0.052243,-0.002772,-0.008743,-0.019831,0.008743,-0.045846,-0.171656,-0.112376,-0.143509,-0.120905,-0.083376,-0.061626,-0.100861,-0.088493,-0.078258,-0.062905,-0.080817,-0.063331
3,-0.134126,-0.107258,-0.096170,-0.097023,-0.117067,-0.099582,-0.099155,-0.100435,-0.083376,-0.098302,-0.118773,-0.105126,-0.098302,-0.044567,-0.040728,-0.050111,-0.065890,-0.069729,-0.038169,-0.020258,-0.083376,-0.065464,-0.065890,-0.008316,-0.048405,-0.028787,-0.060773,-0.051817,-0.063758,-0.053096,-0.063758,-0.045846,-0.044140,-0.036037,-0.005757,0.005331,-0.000640,-0.001493,0.004904,0.012581,...,0.011728,0.044140,-0.010022,0.084229,0.208333,0.221980,0.149053,-0.000213,0.051390,0.108538,-0.007463,-0.084655,-0.240745,-0.410908,-0.411761,-0.436923,-0.432658,-0.417305,-0.411761,-0.397261,-0.366128,-0.302157,-0.292775,-0.130288,-0.045846,-0.098729,0.052670,0.004052,0.056934,0.125170,0.293201,0.219421,-0.199377,0.298319,0.161421,0.095317,-0.155876,-0.081670,-0.126023,-0.032199
4,0.009596,0.047125,0.021963,0.039449,0.078258,0.131994,0.020684,0.096170,-0.032625,-0.064184,-0.009596,-0.036464,-0.082523,-0.156303,-0.100435,-0.201083,-0.134979,-0.334996,-0.230509,-0.204494,-0.183171,-0.191274,-0.282113,-0.214730,-0.252686,-0.204494,-0.244583,-0.244583,-0.146920,-0.256951,-0.180185,-0.202362,-0.175921,-0.130714,-0.044140,-0.142656,-0.147773,-0.058640,-0.093611,-0.009596,...,0.056508,0.088067,-0.018125,0.033052,0.013860,0.026228,-0.023669,0.009596,-0.028787,-0.008743,-0.078685,0.096596,0.104273,-0.065464,0.182744,-0.041155,0.253113,0.034331,-0.274863,-0.167391,-0.128156,0.160568,0.163979,-0.0

In [ ]:
pp_X_sub_482_rshp_df['sleep_stage']=y_sub_482

In [ ]:
pp_X_sub_482_rshp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,sleep_stage
0,-0.236906,-0.234348,-0.218142,-0.187435,-0.143082,-0.184024,-0.207053,-0.213024,-0.187009,-0.258230,-0.239465,-0.255245,-0.197244,-0.247568,-0.243304,-0.288936,-0.264627,-0.252686,-0.224965,-0.231362,-0.326466,-0.266760,-0.224965,-0.231789,-0.321348,-0.260363,-0.269745,-0.236054,-0.228803,-0.259510,-0.398967,-0.433085,-0.476159,-0.439482,-0.430526,-0.419437,-0.386172,-0.360584,-0.346937,-0.335849,...,-0.044140,-0.023243,-0.036037,-0.033478,0.050537,0.065037,0.020684,0.055655,0.043714,0.129861,0.042008,0.088067,0.100861,0.161847,0.123891,0.052670,0.092332,0.002346,0.081670,0.092758,0.110244,0.033478,0.045420,0.120905,-0.000213,0.012581,0.042008,-0.021537,-0.005331,0.036464,0.134979,0.138817,0.049258,0.100861,0.046272,0.065037,0.008743,-0.012155,0.018125,0
1,0.085082,0.021110,0.123464,0.145215,0.168244,0.133273,0.203641,0.077832,0.107258,0.140097,0.189568,0.091905,0.181891,0.143082,0.151185,0.194686,0.251407,-0.022816,0.083376,0.144362,-0.019405,-0.000213,0.049258,0.088067,0.101714,-0.020684,-0.009169,0.031772,-0.000213,-0.021963,-0.059920,-0.009596,-0.009169,-0.003199,0.030066,0.053096,0.060773,0.017699,0.070155,0.114935,...,0.193406,0.164406,0.147347,0.240745,0.167391,0.169097,0.219421,0.253539,0.282966,0.194259,0.225392,0.224112,0.277422,0.278274,0.297892,0.256951,0.268892,0.321348,0.268039,0.268039,0.404937,0.175494,0.184877,0.080817,-0.077405,-0.132420,-0.193833,-0.084655,-0.110670,-0.075699,0.056934,0.222833,0.234348,0.151185,0.167818,0.214303,0.251833,0.268039,0.275289,0
2,0.242451,0.268466,0.242024,0.272730,0.204494,0.171656,0.211318,0.180185,0.187009,0.182318,0.187862,0.150332,0.161421,0.206200,0.166965,0.145215,0.123891,0.088493,0.026228,0.054375,-0.031346,-0.094038,-0.346084,-0.320922,-0.319216,-0.333716,-0.317084,-0.302584,-0.288936,-0.277422,-0.275289,-0.259083,-0.239039,-0.212171,-0.205347,-0.173362,-0.151185,-0.117920,-0.089773,-0.041581,...,0.037317,0.060773,0.040302,0.039875,0.056508,0.045420,0.048831,0.091905,0.109391,0.122185,0.059067,0.149479,0.115361,0.118347,0.133273,0.118347,0.085082,0.067170,0.071861,0.059067,0.063758,0.052243,-0.002772,-0.008743,-0.019831,0.008743,-0.045846,-0.171656,-0.112376,-0.143509,-0.120905,-0.083376,-0.061626,-0.100861,-0.088493,-0.078258,-0.062905,-0.080817,-0.063331,0
3,-0.134126,-0.107258,-0.096170,-0.097023,-0.117067,-0.099582,-0.099155,-0.100435,-0.083376,-0.098302,-0.118773,-0.105126,-0.098302,-0.044567,-0.040728,-0.050111,-0.065890,-0.069729,-0.038169,-0.020258,-0.083376,-0.065464,-0.065890,-0.008316,-0.048405,-0.028787,-0.060773,-0.051817,-0.063758,-0.053096,-0.063758,-0.045846,-0.044140,-0.036037,-0.005757,0.005331,-0.000640,-0.001493,0.004904,0.012581,...,0.044140,-0.010022,0.084229,0.208333,0.221980,0.149053,-0.000213,0.051390,0.108538,-0.007463,-0.084655,-0.240745,-0.410908,-0.411761,-0.436923,-0.432658,-0.417305,-0.411761,-0.397261,-0.366128,-0.302157,-0.292775,-0.130288,-0.045846,-0.098729,0.052670,0.004052,0.056934,0.125170,0.293201,0.219421,-0.199377,0.298319,0.161421,0.095317,-0.155876,-0.081670,-0.126023,-0.032199,0
4,0.009596,0.047125,0.021963,0.039449,0.078258,0.131994,0.020684,0.096170,-0.032625,-0.064184,-0.009596,-0.036464,-0.082523,-0.156303,-0.100435,-0.201083,-0.134979,-0.334996,-0.230509,-0.204494,-0.183171,-0.191274,-0.282113,-0.214730,-0.252686,-0.204494,-0.244583,-0.244583,-0.146920,-0.256951,-0.180185,-0.202362,-0.175921,-0.130714,-0.044140,-0.142656,-0.147773,-0.058640,-0.093611,-0.009596,...,0.088067,-0.018125,0.033052,0.013860,0.026228,-0.023669,0.009596,-0.028787,-0.008743,-0.078685,0.096596,0.104273,-0.065464,0.182744,-0.041155,0.253113,0.034331,-0.274863,-0.167391,-0.128156,0.160568,0.163979,-0.021963,-0.046699,0.299598,0.2283

In [ ]:
pp_x_rshp_df.to_csv(r'/gdrive/My Drive/filtered_seventy_four_subs.csv', header=False, index=False)

In [ ]:
pp_X_sub_482_rshp_df.to_csv(r'/gdrive/My Drive/filtered_x_sub_482.csv', header=False, index=False)

In [ ]:
Fs=100
#batch_size=10

30 epochs

In [ ]:
checkpoint_path_erfan_model = "/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"

In [ ]:
checkpoint_path_erfan_model_dir = os.path.dirname(checkpoint_path_erfan_model)

In [ ]:
cp_callback_erfan_model_30_epochs = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_erfan_model,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=30)

In [ ]:
x_shuf3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
4659,0.012246,0.018036,0.009129,-0.004231,-0.020262,-0.020708,-0.027833,-0.039411,-0.059451,-0.067467,-0.071029,-0.086170,-0.100421,-0.109327,-0.118234,-0.123132,-0.136046,-0.146734,-0.145844,-0.148070,-0.143172,-0.146289,-0.148516,-0.134711,-0.126695,-0.118234,-0.102647,-0.090623,-0.066131,-0.059896,-0.063013,-0.076373,-0.070139,-0.068357,-0.075482,-0.072365,-0.063459,-0.050990,-0.050544,-0.049208,...,-0.023825,-0.046091,-0.023825,-0.003340,0.009129,0.008238,-0.009574,-0.026051,-0.043864,-0.032286,-0.009574,-0.002004,-0.007348,-0.004231,-0.006903,-0.018926,-0.020262,-0.005567,0.012246,0.023380,0.022934,0.030505,0.019817,0.015809,0.024270,0.029169,0.032731,0.034513,0.012246,0.010910,0.011801,0.023380,0.029169,0.036294,0.031841,0.037185,0.032286,0.036739,0.044755,0.066131
83598,-0.011252,0.008653,0.001603,-0.019546,-0.037378,0.000774,-0.009593,0.019850,0.000359,-0.030743,-0.030743,-0.020790,0.007409,-0.085482,-0.029499,-0.030328,-0.040281,-0.039866,-0.040695,-0.042769,-0.026181,-0.064333,-0.059357,-0.043183,-0.017887,-0.000470,-0.028255,-0.040281,-0.015399,-0.039451,-0.014570,-0.022449,-0.019131,-0.018717,-0.038622,-0.029499,0.019021,-0.044428,-0.034890,0.012385,...,-0.057283,-0.009593,-0.045257,-0.027010,-0.013326,-0.056039,-0.028669,-0.012081,-0.045257,-0.056868,-0.043183,-0.031572,-0.008764,-0.003373,-0.033646,0.003262,0.031461,0.036852,0.052611,0.012385,0.034364,0.048049,0.047634,0.028973,-0.015399,0.001603,0.032705,0.029803,-0.003788,0.067540,0.081225,-0.010837,0.036023,-0.021619,0.058002,0.036438,0.050123,0.021094,-0.000470,0.008238
96670,0.083314,0.118263,0.099761,0.100994,0.088660,0.120730,0.105928,0.089071,0.096060,0.095649,0.105106,0.060290,0.086193,0.029453,0.034798,-0.001795,-0.027698,-0.038389,-0.074982,-0.113220,-0.132133,-0.120209,-0.118976,-0.114864,-0.114042,-0.113631,-0.103352,-0.095540,-0.086494,-0.069225,-0.086494,-0.062647,-0.026465,-0.003029,0.016296,0.003961,0.013006,0.050833,0.032331,0.005605,...,0.022463,0.041376,0.031097,-0.000562,0.021229,-0.006729,0.006839,0.002316,0.000260,0.004783,0.009717,0.035620,0.058234,0.049599,0.078792,0.064401,0.056589,0.069335,0.076736,0.031097,0.038909,-0.016597,-0.017831,-0.076215,-0.101296,-0.138711,-0.158858,-0.215187,-0.232867,-0.264526,-0.312632,-0.308520,-0.328256,-0.344291,-0.330311,-0.347169,-0.298241,-0.327844,-0.303997,-0.296596
45266,-0.006153,0.002181,0.012830,0.021626,0.041072,0.053573,0.051721,0.053573,0.053110,0.054962,0.043387,0.042461,0.030423,0.024867,0.024404,0.019312,0.022552,0.022552,0.026719,0.015145,-0.002912,-0.004764,-0.008005,-0.017728,-0.024673,-0.022821,-0.032081,-0.029303,-0.036248,-0.042267,-0.047360,-0.058472,-0.056620,-0.053379,-0.043656,-0.042730,-0.031155,-0.022358,-0.000134,0.009126,...,-0.014950,-0.013561,-0.018191,-0.009857,-0.020506,-0.021432,-0.012172,-0.014487,-0.017265,-0.025599,-0.025136,-0.016339,-0.008005,-0.009394,-0.014487,-0.007079,-0.003838,0.004959,0.005422,-0.000134,0.002644,-0.000134,0.010515,0.003570,0.007737,0.016534,0.013293,0.012367,0.017460,0.021626,0.030886,0.033201,0.022552,0.018386,0.024404,0.019775,0.009589,-0.000134,0.009126,0.016071
71553,-0.052897,-0.055795,-0.033093,-0.021500,0.002168,0.016658,-0.023916,-0.032610,-0.047101,-0.033093,-0.032610,-0.076565,-0.112309,-0.112309,-0.097335,-0.066905,-0.076082,-0.084776,-0.063040,-0.052897,-0.072701,-0.072218,-0.075116,-0.077048,-0.071252,-0.051931,-0.077048,-0.107479,-0.104580,-0.097335,-0.103614,-0.103614,-0.103131,-0.078980,-0.064006,-0.068354,-0.062074,-0.049033,-0.000730,0.031149,...,-0.000247,-0.007493,-0.022466,-0.037923,-0.027780,-0.063523,-0.014738,-0.012323,-0.008459,-0.013772,-0.035025,-0.027780,0.016175,0.028251,0.043225,0.066893,0.081384,0.095874,

In [ ]:
model_30_epochs = model_b(verbose=VBS)
hist_30_epochs = model_30_epochs.fit(
    x_shuf3_rshp, y_shuf3_, batch_size=batch_size, epochs=30, verbose=VBS, callbacks=[cp_callback_erfan_model_30_epochs]
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
y_pred_shuf3 = model_30_epochs.predict(pp_X_sub_482, batch_size=batch_size)
y_pred_shuf3 = np.array([np.argmax(s) for s in y_pred_shuf3])
f1_cnn_shuf3 = f1_score(y_sub_482, y_pred_shuf3, average="macro")
if VBS:
    print("F1 score: {}".format(f1_cnn_shuf3))
    report_shuf3 = classification_report(y_sub_482, y_pred_shuf3)
    print(report_shuf3)

F1 score: 0.7331929129755452
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1351
           1       0.45      0.14      0.21       187
           2       0.74      0.94      0.83       807
           3       0.89      0.92      0.91       318
           4       0.82      0.69      0.75       407

    accuracy                           0.86      3070
   macro avg       0.78      0.73      0.73      3070
weighted avg       0.85      0.86      0.85      3070



30 epochs

F1 score: 0.7331929129755452

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1351
           1       0.45      0.14      0.21       187
           2       0.74      0.94      0.83       807
           3       0.89      0.92      0.91       318
           4       0.82      0.69      0.75       407

    accuracy                           0.86      3070
    macro avg       0.78      0.73      0.73      3070
    weighted avg       0.85      0.86      0.85      3070
   



   



  
   


50 subjects

30 epochs: 86%

In [ ]:
latest_30_epochs=tf.train.latest_checkpoint(checkpoint_path_erfan_model_dir)

In [ ]:
#print(latest_30_epochs)
epochs_30_model_path="/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"

In [ ]:
loaded_model_30_epochs = model_b(verbose=VBS)
loaded_model_30_epochs.load_weights(epochs_30_model_path)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_30_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_sub_482_=np_utils.to_categorical(y_sub_482)

In [ ]:
y_sub_482_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
accuracy_loaded_model_30_epochs = loaded_model_30_epochs.evaluate(pp_X_sub_482, y_sub_482_)

96/96 [==============================] - 2s 4ms/step - loss: 0.4543 - accuracy: 0.8632
